In [4]:
from btc_streamer.ml.preprocessing import BTCDataloader
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from xgboost.spark import SparkXGBClassifier
import re
import logging

# Initialize logging
logging.basicConfig(level=logging.INFO)

# Initialize Spark session
spark = SparkSession.builder \
.appName("XGBoostSparkTest") \
.getOrCreate()
logging.info("Spark session initialized successfully.")

# Load and prepare data
btc = BTCDataloader()
btc.setup_spark()
df = btc.load_data(directory='../data/')
train, test, spark_session = btc.preproc_split(df)

24/09/26 13:46:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
INFO:root:Spark session initialized successfully.
24/09/26 13:46:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


train: 73770, test: 18226


In [15]:
xgb = SparkXGBClassifier(features_col= "features", 
                        label_col="target", 
                        num_workers = 4, 
                        random_state = 1,
                        missing = None, 
                        validation_indicator_col = 'isVal',
                        eval_metric = 'aucpr'    
                       )

In [16]:
xgb.setParams(early_stopping_rounds=5)

In [17]:
xgb.__dict__

{'_input_kwargs': {'features_col': 'features',
  'label_col': 'target',
  'num_workers': 4,
  'random_state': 1,
  'missing': None,
  'validation_indicator_col': 'isVal',
  'eval_metric': 'aucpr'},
 'uid': 'SparkXGBClassifier_6145cfee0a33',
 '_paramMap': {Param(parent='SparkXGBClassifier_6145cfee0a33', name='featuresCol', doc='features column name.'): 'features',
  Param(parent='SparkXGBClassifier_6145cfee0a33', name='labelCol', doc='label column name.'): 'target',
  Param(parent='SparkXGBClassifier_6145cfee0a33', name='num_workers', doc='The number of XGBoost workers. Each XGBoost worker corresponds to one spark task.'): 4,
  Param(parent='SparkXGBClassifier_6145cfee0a33', name='random_state', doc='Refer to XGBoost doc of xgboost.sklearn.XGBClassifier for this param random_state'): 1,
  Param(parent='SparkXGBClassifier_6145cfee0a33', name='missing', doc='Refer to XGBoost doc of xgboost.sklearn.XGBClassifier for this param missing'): None,
  Param(parent='SparkXGBClassifier_6145cfee0a3

In [19]:
train.show(5)

+--------------------+------+
|            features|target|
+--------------------+------+
|[-0.0935832341819...|     0|
|[-0.0800532148741...|     0|
|[-0.0697162404845...|     0|
|[-0.0660354893981...|     0|
|[-0.0650725049758...|     0|
+--------------------+------+
only showing top 5 rows



In [18]:
model = xgb.fit(train)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `isVal` cannot be resolved. Did you mean one of the following? [`target`, `features`].;
'Project [target#395 AS label#496, UDF(features#425) AS values#501, 'isVal AS validationIndicator#502]
+- Sample 0.0, 0.8, false, 42
   +- Sort [features#425 ASC NULLS FIRST, target#395 ASC NULLS FIRST], false
      +- Sample 0.0, 0.2, false, 42
         +- Sort [features#425 ASC NULLS FIRST, target#395 ASC NULLS FIRST], false
            +- Project [features#425, target#395]
               +- Project [timestamp#324, price#316, Volume_USD#325, percent_change_30m#340, percent_change_1h#349, percent_change_6h#359, percent_change_12h#370, percent_change_24h#382, target#395, UDF(struct(percent_change_30m, percent_change_30m#340, percent_change_1h, percent_change_1h#349, percent_change_6h, percent_change_6h#359, percent_change_12h, percent_change_12h#370, percent_change_24h, percent_change_24h#382)) AS feature

In [2]:
# Convert feature columns into a single vector column
feature_columns = [x.name for x in df.schema if re.search(r'percent', x.name)]
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features') 

assembler

VectorAssembler_c6f9bd947d95

In [3]:
assembled_data = assembler.transform(df).select(['features', 'target'])

In [4]:
train_data, test_data = assembled_data.randomSplit([0.8, 0.2], seed=42)

In [1]:
train_data.show(5)

NameError: name 'train_data' is not defined

In [5]:
params = {
        'objective': 'binary:logistic',  # Binary classification
        'booster': 'gbtree',
        'tree_method': 'auto',           # Ensure using CPU
        'num_round': 100,
        'eta': 0.1
    }

In [7]:
xgb_estimator = SparkXGBClassifier(
        features_col='features',
        label_col='target',use_gpu=False
    )

In [8]:
train_data.show(5)

24/09/25 12:38:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 1

+--------------------+------+
|            features|target|
+--------------------+------+
|[-0.1015825605186...|     0|
|[-0.0805882532746...|     0|
|[-0.0750382016173...|     0|
|[-0.0712359669925...|     0|
|[-0.0697370117538...|     0|
+--------------------+------+
only showing top 5 rows



In [9]:
model = xgb_estimator.fit(train_data)

24/09/25 12:38:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 12:38:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 1

Py4JError: An error occurred while calling None.org.apache.spark.api.python.PythonFunction. Trace:
py4j.Py4JException: Constructor org.apache.spark.api.python.PythonFunction([class [B, class java.util.HashMap, class java.util.ArrayList, class java.lang.String, class java.lang.String, class java.util.ArrayList, class org.apache.spark.api.python.PythonAccumulatorV2]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:180)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:197)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:842)



In [27]:
model

NameError: name 'model' is not defined